## 5.3 기술 통계 계산과 요약

* pandas 객체는 일반적인 수학 메서드와 통계 메서드를 가지고 있다. 이 메서드의 대부분은 하나의 Series나 DataFrame의 로우나 컬럼에서 단일 값(합이나 평균 같은)을 구하는 축소(reductions) 혹은 요약 통계(summary statistics) 범주에 속한다. 순수 NumPy 배열에서 제공하는 동일한 메서드와 비교하여 pandas의 메서드는 처음부터 누락된 데이터를 제외하도록 설계되었다.

In [12]:
import pandas as pd
import numpy as np

In [13]:
df = pd.DataFrame([[1.4, np.nan], [7.1, -4.5],
                   [np.nan, np.nan], [0.75, -1.3]],
                  index = ['a', 'b', 'c', 'd'],
                  columns = ['one', 'two'])
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


DataFrame의 sum 메서드를 호출하면 각 컬럼의 합을 담은 Series를 반환한다.

In [14]:
df.sum()

one    9.25
two   -5.80
dtype: float64

axis = 'columns' 또는 axis = 1 옵션을 넘기면 각 컬럼의 합을 반환한다.

In [15]:
df.sum(axis = 'columns')

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

전체 로우나 컬럼의 값이 NA가 아니라면 NA 값은 제외되고 계산된다. 이는 Skipna 옵션으로 조정할 수 있다.

In [16]:
df.mean(axis = 'columns', skipna=False)

a      NaN
b    1.300
c      NaN
d   -0.275
dtype: float64

**축소 메서드의 옵션**

**axis** : 연산을 수행할 축, DataFrame에서 0은 로우고, 1은 컬럼이다.

**skipna** : 누락된 값을 제외할 것인지 정하는 옵션. 기본값은 True

**level** : 계산하려는 축이 계층적 색인(다중 색인)이라면 레벨에 따라 묶어서 계산한다.


idxmin이나 idxmax 같은 메서드는 최솟값 혹은 최댓값은 가지고 있는 색인값과 같은 간접 통계를 반환한다.

In [17]:
df.idxmax()

one    b
two    d
dtype: object

또 다른 메서드로 누산 (accumulation)이 있다.

In [18]:
df.cumsum()

,one,two
a,1.40,NaN
b,8.50,-4.5
c,NaN,NaN
d,9.25,-5.8


describe : 한 번에 여러 개의 통계 결과를 만들어낸다.

In [19]:
df.describe()

,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,1.075000,-3.700000
50%,1.400000,-2.900000
75%,4.250000,-2.100000
max,7.100000,-1.300000


수식 데이터가 아닐 경우 describe는 다른 요약 통계를 생성한다.

In [20]:
obj = pd.Series(['a', 'a', 'b', 'c'] * 4)
obj.describe()

count     16
unique     3
top        a
freq       8
dtype: object

**요약 통계 관련 메서드**

**count** : NA 값을 제외한 값의 수를 반환한다.

**describe** : Series나 DataFrame의 각 컬럼에 대한 요약 통계를 계산한다.

**min, max** : 최솟값과 최댓값을 계산한다.

**argmin, argmax** : 각각 최솟값과 최댓값을 담고 있는 색인의 위치(정수)를 반환한다.

**idxmin, idxmax** : 각각 최솟값과 최댓값을 담고 있는 색인의 값을 반환한다.

**quantile** : 0부터 1까지의 분위수를 계산한다.

**sum** : 합을 계산한다.

**mean** : 평균을 계산한다.

**median** : 중간값(50% 분위)을 반환한다.

**mad** : 평균값에서 평균절대편차를 계산한다.

**prod** : 모든 값의 곱

**var** : 표본분산의 값을 계산한다,

**std** : 표본표춘편차의 값을 계산한다.

**skew** : 표본비대칭도(3차 적률)의 값을 계산한다.

**kurt** : 표본첨도(4차 적률)의 값을 계산한다.

**cumsum** : 누적합을 계산한다.

**cummin, cummax** : 각각 누적 최솟값과 누적 최댓값을 계산한다.

**cumprod** : 누적곱을 계산한다

**diff** : 1차 산술자를 계산한다. (시계열 데이터 처리 시 유용하다.)

**pct_change** : 퍼센트 변화율을 계산한다.


### 5.3.1 상관관계와 공분사

상관관계나 공분산 가튼 요약 통계 계산은 두 쌍의 인자를 필요로 한다. pandas-datareader 패키지를 이용해서 야후! 금융 사이트에서 구한 주식가격과 시가총액을 담고 있는 다음 DataFrame을 생각해본다.

In [21]:
!pip install pandas-datareader

In [22]:
!pip install yfinance


In [23]:
import yfinance as yf

# 종목 리스트
tickers = ['AAPL', 'IBM', 'MSFT', 'GOOG']

# 데이터 가져오기
all_data = {ticker: yf.download(ticker, start='2020-01-01', end='2021-01-01')
            for ticker in tickers}

# 데이터프레임 생성
price = pd.DataFrame({ticker: data['Adj Close'] for ticker, data in all_data.items()})
volume = pd.DataFrame({ticker: data['Volume'] for ticker, data in all_data.items()})


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


각 주식의 퍼센트 변화율 계산

In [24]:
returns = price.pct_change()

In [25]:
returns.tail()

,AAPL,IBM,MSFT,GOOG
Date,,,,
2020-12-24,0.007712,0.006376,0.007827,0.003735
2020-12-28,0.035766,0.001043,0.009921,0.021416
2020-12-29,-0.013315,-0.008172,-0.003601,-0.009780
2020-12-30,-0.008527,0.004362,-0.011020,-0.010917
2020-12-31,-0.007703,0.012386,0.003338,0.007105


corr 메서드는 NA가 아니며 정렬된 새긴에서 연속하는 두 Series에 대해 상관관계를 계산하고 cov 메서드는 공분산을 계산한다.

In [26]:
returns['MSFT'].corr(returns['IBM'])

0.6400916481227401

In [27]:
returns['MSFT'].cov(returns['IBM'])

0.0004568363080745354

MSFT는 파이썬 속성 이름 규칙에 어긋나지 않으므로 좀 더 편리한 문법으로 해당 컬럼을 선택할 수 있다.

In [29]:
returns.MSFT.corr(returns.IBM)

0.6400916481227401

반면에 DataFrame에서 corr과 cov 메서드는 DataFrame 행렬에서 상관관계와 공분산을 계산한다.

In [30]:
returns.corr()

,AAPL,IBM,MSFT,GOOG
AAPL,1.000000,0.575533,0.839273,0.751909
IBM,0.575533,1.000000,0.640092,0.657524
MSFT,0.839273,0.640092,1.000000,0.852444
GOOG,0.751909,0.657524,0.852444,1.000000


In [31]:
returns.cov()

,AAPL,IBM,MSFT,GOOG
AAPL,0.000865,0.000437,0.000683,0.000534
IBM,0.000437,0.000665,0.000457,0.000409
MSFT,0.000683,0.000457,0.000766,0.000570
GOOG,0.000534,0.000409,0.000570,0.000583


DataFrame의 corrwith 메서드를 사용하면 다른 Series나 DataFrame과의 상관관계를 계산한다. Series를 넘기면 각 컬럼에 대해 계산한 상관관계를 담고 있는 Series를 반환한다.

In [32]:
returns.corrwith(returns.IBM)

AAPL    0.575533
IBM     1.000000
MSFT    0.640092
GOOG    0.657524
dtype: float64

DataFrame을 넘기면 맞아떨어지는 컬럼 이름에 대한 상관관계를 계산한다. 여기서 나는 시가총액의 퍼센트 변화율에 대한 상관관계 계산

In [33]:
returns.corrwith(volume)

AAPL   -0.059310
IBM    -0.045344
MSFT   -0.070795
GOOG   -0.110356
dtype: float64

axis = 'columns' 옵션을 넘기면 각 컬럼에 대한 상관관계의 공분산을 계산한다. 모든 경우 데이터는 상관관계를 계산하기 위해 이름순으로 정렬

### 5.3.2 유일값, 값 세기, 멤버십

또 다른 종류의 메서들는 1차원 Series에 담긴 정보를 추출하는 메서드가 있다.

In [35]:
obj = pd.Series(['c', 'a', 'd', 'a', 'a', 'b', 'b', 'c', 'c'])

unique 메서드는 중복되는 값을 제거하고 유일값만 담고 있는 Series를 반환한다.

In [37]:
uniques = obj.unique()
uniques

array(['c', 'a', 'd', 'b'], dtype=object)

유일값은 정렬된 순서대로 반환되지 않지만 필요하다면 uniques.sort()를 이용해서 나중에 정렬할 수 있다.

그리고 value_counts 메서드는 Series에서 도수(frequency)를 계산하여 반환한다.

In [38]:
obj.value_counts()

c    3
a    3
b    2
d    1
dtype: int64

value_count에서 반환하는 Series는 담고 있는 값을 내림차순으로 정렬한다. value_counts 메서드는 pandas의 최상위 메서드로, 어떤 배열이나 순차 자료구조에서도 사용할 수 있다.

In [40]:
pd.value_counts(obj.values, sort=False)

c    3
a    3
d    1
b    2
dtype: int64

isin 메서드는 어떤 값이 Series에 존재하는지 나타내는 불리언 벡터를 반환하는데, Series나  DataFrame의 컬럼에서 값을 골라내고 싶을 때 유용하게 사용할 수 있다.

In [41]:
obj

0    c
1    a
2    d
3    a
4    a
5    b
6    b
7    c
8    c
dtype: object

In [45]:
mask = obj.isin(['b', 'c'])
mask

0     True
1    False
2    False
3    False
4    False
5     True
6     True
7     True
8     True
dtype: bool

In [46]:
obj[mask]

0    c
5    b
6    b
7    c
8    c
dtype: object

isin과 관련이 있는 Index.get_indexer 메서드는 여러 값이 들어 있는 배열에서 유일한 값의 색인 배열을 구할 수 있다.

In [47]:
to_match = pd.Series(['c', 'a', 'b', 'b', 'c', 'a'])

In [48]:
unique_vals = pd.Series(['c', 'b', 'a'])

In [49]:
pd.Index(unique_vals).get_indexer(to_match)

array([0, 2, 1, 1, 0, 2])

**유일값, 값 세기, 멤버십 메서드**

**isin** : Series의 각 원소가 넘겨받은 연속된 값에 속하는지 나타내는 불리언 배열을 반환한다.

**match** : 각 값에 대해 유일한 값을 담고 있는 배열에서의 정수 색인을 계산한다. 데이터 정렬이나 조인 형태의 연산을 하는 경우에 유용하다.

**unique** : Series에서 중복되는 값을 제거하고 유일값만 포함하는 배열을 반환한다. 결과는 Series에서 발견된 순서대로 반환한다.

**value_counts** : Series에서 유일값에 대한 색인과 도수를 계산한다. 도수는 내림차순으로 정렬된다.


DataFrame의 여러 컬럼에 대해 히스토그램을 구해야 하는 경우가 있다.

In [51]:
data = pd.DataFrame({'Qu1' : [1, 3, 4, 3, 4],
                      'Qu2' : [2, 3, 1, 2, 3],
                      'Qu3' : [1, 5, 2, 4, 4]})
data

,Qu1,Qu2,Qu3
0,1,2,1
1,3,3,5
2,4,1,2
3,3,2,4
4,4,3,4


위 DataFrame의 apply 함수에 pandas.value_counts를 넘기면 다음과 같은 결과를 얻을 수 있다.

In [52]:
result = data.apply(pd.value_counts).fillna(0)
result

,Qu1,Qu2,Qu3
1,1.0,1.0,1.0
2,0.0,2.0,1.0
3,2.0,2.0,0.0
4,2.0,0.0,2.0
5,0.0,0.0,1.0


결과값의 로우 라벨은 전체 컬럼의 유일한 값들은 담고 있다. 각 값은 각 컬럼에서 해당값이 몇 번 출현했는지 나타낸다.